In [1]:
import torch
import numpy
import pennylane as qml
import pennylane.numpy as np
import gymnasium as gym
import matplotlib.pyplot as plt

In [2]:
def Cliff2():
    """
    Random 2-qubit Clifford circuit.

    Arguments:
        -nodes (np.ndarray): 
    
    Returns:
        -null
    """
    
    weights = np.random.randint(2, size=(2, 10))
    
    return qml.matrix(qml.RandomLayers(weights=weights,wires=[0,1])).numpy()

In [3]:
def RandomLayers(N_QUBITS, DEPTH):
    """
    Generates brick wall pattern of random 2 qubit Clifford gates

    Arguments:
        -N_QUBITS (int): Number of qubits
        -DEPTH (int): Depth of the circuit

    Returns:
        -random_layers (np.ndarray): Array of 4x4 unitaries (N_QUBITS, DEPTH, 4, 4)
    
    """

    random_layers = []
    for t in range(DEPTH):
        layer = []
        for x in range(0,N_QUBITS,2):
                layer.append(Cliff2())
        random_layers.append(layer)

    return random_layers


In [4]:
N_QUBITS = 2*2
DEPTH = 3

# random_layers = []
# # for t in range(DEPTH):
# #         layer = []
# #         for x in range(0,N_QUBITS,2):
# #                 layer.append(Cliff2())
# #         random_layers.append(layer)

random_layers = RandomLayers(N_QUBITS,DEPTH)


dev = qml.device("default.qubit", wires=N_QUBITS)

@qml.qnode(dev)
def circuit(theta):
    """
    Quantum circuit with random entangling Clifford layers and disentangling layers.
    
    Arguments:
        -theta (np.ndarray): Binary matrix representing the positions of projections. (N_QUBITS, DEPTH)
    
    Returns:
        -Average Von Neumann entropy (float32): Average of 2-qubit Von Neumann entropies over all neighbors.
    """

    theta = theta.T
    DEPTH,N_QUBITS = np.shape(theta)

    for t in range(DEPTH):
        layer = random_layers[t]
        if t%2==0:
            for x in range(0,N_QUBITS,2):
                brick = layer[int(x/2)]
                qml.QubitUnitary(brick,wires=[x,x+1])
        elif t%2==1:
            for x in range(1,N_QUBITS-2,2):
                brick = layer[int((x-1)/2)]
                qml.QubitUnitary(brick,wires=[x,x+1])
            brick = layer[-1]
            qml.QubitUnitary(brick,wires=[N_QUBITS-1,0])
            
        projections = theta[t]
        for x in range(N_QUBITS):
            if projections[x]==1:
                qml.Projector(state=[0],wires=[x])
            
    entropies = []
    for x in range(N_QUBITS-1):
        entropies.append(qml.vn_entropy(wires=[x,x+1]))
    entropies.append(qml.vn_entropy(wires=[N_QUBITS-1,0]))
        
    return entropies

In [5]:
random_layers[0][1]

array([[-0.2248451 -0.07056j   ,  0.        -0.42073549j,
        -0.03812373+0.76744761j,  0.41652498+0.0593742j ],
       [-0.17508774+0.3825737j ,  0.17508774-0.3825737j ,
         0.35403671-0.22732436j, -0.06211013+0.68197307j],
       [ 0.07056   -0.7651474j , -0.42073549+0.j        ,
         0.23255239-0.03812373j, -0.0593742 +0.41652498j],
       [ 0.3825737 +0.17508774j,  0.15772861+0.66638324j,
         0.22732436+0.35403671j,  0.22732436+0.35403671j]])

In [6]:
theta = np.random.randint(2, size=(N_QUBITS,DEPTH))
print(circuit(theta))
drawer = qml.draw(circuit)

print(drawer(theta))

[1.1102230246251564e-16, 1.1102230246251564e-16, 1.1102230246251564e-16, 1.1102230246251564e-16]
0: ─╭U(M0)──|0⟩⟨0|────────╭U(M3)──|0⟩⟨0|─╭U(M4)──|0⟩⟨0|─┤ ╭vnentropy                      
1: ─╰U(M0)──|0⟩⟨0|─╭U(M2)─│──────────────╰U(M4)──|0⟩⟨0|─┤ ╰vnentropy ╭vnentropy           
2: ─╭U(M1)─────────╰U(M2)─│───────|0⟩⟨0|─╭U(M5)──|0⟩⟨0|─┤            ╰vnentropy ╭vnentropy
3: ─╰U(M1)──|0⟩⟨0|────────╰U(M3)─────────╰U(M5)──|0⟩⟨0|─┤                       ╰vnentropy

  ╭vnentropy
  │         
  │         
  ╰vnentropy

M0 = 
[[ 0.13289575+0.48042655j  0.03661081-0.17596205j -0.13914256+0.4364598j
   0.71300901-0.03140716j]
 [ 0.17596205+0.03661081j  0.43341148+0.09307212j  0.03140716+0.71300901j
  -0.47917584+0.17947517j]
 [-0.72348262-0.24839441j -0.36460155-0.03413046j  0.13289575+0.48042655j
   0.03661081-0.17596205j]
 [ 0.03413046-0.36460155j  0.21982151+0.76731245j  0.17596205+0.03661081j
   0.43341148+0.09307212j]]
M1 = 
[[-0.2248451 -0.07056j     0.        -0.42073549j -0.03812373+0.76744

In [7]:
def RandomFlip(theta,K):
    """
    Randomly flip K entries of a binary matrix theta.

    Arguments:
        -theta (np.ndarray):
        -K (int): 

    Returns:
        -flipped (np.ndarray):

    """
    
    x,y = np.shape(theta)
    N = int(x*y)
    arr = np.array([0] * (N-K) + [1] * K)
    np.random.shuffle(arr)
    arr = arr.reshape(np.shape(theta))

    flipped = (theta + arr) % 2
    
    return flipped

In [8]:
class Disentangler(gym.Env):
    """
    Reinforcement learning environment for the disentangler.

    """
    
    def __init__(self):
        self.action_space = gym.spaces.Discrete(10)
        self.observation_space = []
        self.state = np.zeros((N_QUBITS,DEPTH))
        self.moves = 10

    def step(self, action):
        self.state = RandomFlip(self.state, action)
        self.moves += -1

        entropies = circuit(self.state)
        entropy = sum(entropies)/len(entropies)
        
        if entropy < 1e-16:
            reward = 100
        elif entropy > 1e-16:
            reward = -1

        if self.moves <= 0:
            done = True
        else:
            done = False

        info = {}
        
        return self.state, reward, done, info
    
    def reset(self):
        self.state = np.zeros((N_QUBITS,DEPTH))
        self.moves = 10

        return self.state

In [9]:
env = Disentangler()

In [23]:
episodes = 50
matrix_labels = {}
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0

    while not done:
        action = env.action_space.sample()
        n_state, reward, done, info = env.step(action)
        score += reward
    
    index = len(matrix_labels)
    label = "m_"+str(index+1)
    if not(label in matrix_labels.values()):
        matrix_labels["m_"+str(index+1)] = n_state
    
    print('Episode:{} Score:{}'.format(episode, score))


Episode:1 Score:-10


ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [24]:
d = {}
d["m_1"] = np.random.randint(2, size=(2,2))
d["m_2"] = np.random.randint(2, size=(2,2))
d["m_3"] = np.random.randint(2, size=(2,2))

In [41]:
matrix = np.tensor([[0, 0],[1, 1]])
d.get(matrix)

TypeError: unhashable type: 'numpy.tensor'

In [22]:
type(np.random.randint(2, size=(2,2)))

pennylane.numpy.tensor.tensor

In [28]:
anan = np.tensor([[0, 0],
        [1, 1]], requires_grad=True)